In [4]:
import sqlite3
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.wait import WebDriverWait
import selenium.webdriver.support.expected_conditions as EC
import os
from urllib.request import urlopen
from lxml import etree


In [77]:
def get_doi_page_link(doi):
    doi_page_link = f"https://www2.bgs.ac.uk/nationalgeosciencedatacentre/citedData/catalogue/{doi.split('/')[-1]}.html"

def get_chrome_driver():
    service = ChromeService(executable_path=ChromeDriverManager().install())
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    options.add_argument("--disable-logging")
    options.add_argument("--window-size=1920,1080")
    options.add_experimental_option('excludeSwitches', ['disable-logging']) # stop noise logging.
    driver = webdriver.Chrome(service=service, options=options)
    return driver


def getfid(d_link):
    prefix = {
        "www2":"https://www2.bgs.ac.uk/nationalgeosciencedatacentre/citedData/catalogue/",
        "metadata": "https://metadata.bgs.ac.uk/geonetwork/srv/eng/catalog.search#/metadata/"
        }
    if d_link is None:
        return None
    if d_link.startswith(prefix['www2']):
        return d_link.replace(prefix['www2'], "").replace(".html", "")
    if d_link.startswith(prefix['metadata']):
        return d_link.replace(prefix['metadata'], "")
def get_discovery_link_by_selenium(doi_url):
    driver = get_chrome_driver()
    print("selenium handling: " + doi_url )
    driver.get(doi_url)
    try:
        link = driver.find_element(By.LINK_TEXT, "Discovery Link")
    except Exception as e:
        print(e)
        print(f"[ERR] {doi_url}")
        driver.close()
        return None
    
    link.click()
    ret = None
    
    if len(driver.window_handles) > 1:
        driver.switch_to.window(driver.window_handles[1])
    
    driver.set_page_load_timeout(10)
    try:
         WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "/html/body/div[2]/footer/div[1]/div[2]/p/a")))
         print("Page is ready!")
         ret = driver.current_url
    except Exception as e:
        print(e)
        print(f"[ERR] {doi_url}")
    finally:
        driver.close()
    
    with open("doi2discovery_link.txt", "a") as f:
        f.write(f"{doi_url} : {ret}\n")
    
    return ret

In [61]:
db_fp = r"C:\Users\ChrisLam\Documents\nerc-digital-solutions-hub\Hub\DotNet\Out\Log\MetadataDB\22-02-2023_metadata.db"
ngdc_table_url = "https://www2.bgs.ac.uk/nationalgeosciencedatacentre/citeddata/catalogue.html"


ngdc_df = pd.read_html(ngdc_table_url)[0]
ngdc_df['DoiPageLink'] = ngdc_df['Digital Object Identifier'].apply(lambda doi: f"https://www2.bgs.ac.uk/nationalgeosciencedatacentre/citedData/catalogue/{doi.split('/')[-1]}.html")
ngdc_df['DiscoveryLink'] = ngdc_df['DoiPageLink'].apply(lambda doi_url: get_discovery_link_by_selenium(doi_url))
ngdc_df['FileIdentifier'] = ngdc_df['DiscoveryLink'].apply(lambda d_link: getfid(d_link))
ngdc_df.to_pickle("ngdc_df.pkl")

db_conn = sqlite3.connect(db_fp)
md_df = pd.read_sql("select * from MetadataEntities", db_conn)
md_df['DC'] = md_df.FileIdentifier.apply(lambda fid: "NGDC" if fid in df.FileIdentifier.tolist() else None)
md_df.to_sql("MetadataEntities", db_conn, if_exists="replace", index=False)
ngdc_df.to_sql("NgdcEntity", db_conn, if_exists="replace", index=False)
db_conn.close()

[WDM] - ====== WebDriver manager ======


2023-02-23 20:48:19,099 INFO ====== WebDriver manager ======


[WDM] - Current google-chrome version is 110.0.5481


2023-02-23 20:48:19,101 INFO Current google-chrome version is 110.0.5481


[WDM] - Get LATEST chromedriver version for 110.0.5481 google-chrome


...

[WDM] - Get LATEST chromedriver version for 110.0.5481 google-chrome


2023-02-23 21:28:21,191 INFO Get LATEST chromedriver version for 110.0.5481 google-chrome


[WDM] - Driver [C:\Users\ChrisLam\.wdm\drivers\chromedriver\win32\110.0.5481.77\chromedriver.exe] found in cache


2023-02-23 21:28:21,280 INFO Driver [C:\Users\ChrisLam\.wdm\drivers\chromedriver\win32\110.0.5481.77\chromedriver.exe] found in cache
selenium handling: https://www2.bgs.ac.uk/nationalgeosciencedatacentre/citedData/catalogue/4124aa40-e77c-4856-b7bf-95daa34cd6f5.html
Page is ready!


385

In [134]:
# https://www2.bgs.ac.uk/nationalgeosciencedatacentre/citedData/catalogue/485a5606-c347-4b72-b8a8-9da82ff239af.html
df = pd.read_pickle("ngdc_df.pkl")
null_dlink_df = df[df.DiscoveryLink.isnull()]
df[df.DiscoveryLink.isnull()]['DiscoveryLinks'] = df[df.DiscoveryLink.isnull()]['DoiPageLink'].apply(lambda doi_url: get_discovery_link_by_selenium(doi_url))

[WDM] - ====== WebDriver manager ======


2023-02-23 22:22:56,206 INFO ====== WebDriver manager ======


[WDM] - Current google-chrome version is 110.0.5481


2023-02-23 22:22:56,211 INFO Current google-chrome version is 110.0.5481


[WDM] - Get LATEST chromedriver version for 110.0.5481 google-chrome


...

[WDM] - Get LATEST chromedriver version for 110.0.5481 google-chrome


2023-02-23 22:31:04,396 INFO Get LATEST chromedriver version for 110.0.5481 google-chrome


[WDM] - Driver [C:\Users\ChrisLam\.wdm\drivers\chromedriver\win32\110.0.5481.77\chromedriver.exe] found in cache


2023-02-23 22:31:04,462 INFO Driver [C:\Users\ChrisLam\.wdm\drivers\chromedriver\win32\110.0.5481.77\chromedriver.exe] found in cache
selenium handling: https://www2.bgs.ac.uk/nationalgeosciencedatacentre/citedData/catalogue/e729d509-a616-4787-bb41-44c1169685b6.html
Message: no such element: Unable to locate element: {"method":"link text","selector":"Discovery Link"}
  (Session info: chrome=110.0.5481.104)
Stacktrace:
Backtrace:
	(No symbol) [0x00AC37D3]
	(No symbol) [0x00A58B81]
	(No symbol) [0x0095B36D]
	(No symbol) [0x0098D382]
	(No symbol) [0x0098D4BB]
	(No symbol) [0x009C3302]
	(No symbol) [0x009AB464]
	(No symbol) [0x009C1215]
	(No symbol) [0x009AB216]
	(No symbol) [0x00980D97]
	(No symbol) [0x0098253D]
	GetHandleVerifier [0x00D3ABF2+2510930]
	GetHandleVerifier [0x00D68EC1+2700065]
	GetHandleVerifier [0x00D6C86C+2714828]
	GetHandleVerifier [0x00B73480+645344]
	(No symbol) [0x00A60FD2]
	(No symbol) [0x00A66C68]
	(No symbol) [0x00A66D4B]
	(No symbol) [0x00A70D6B]
	BaseThreadInitThu

C:\Users\ChrisLam\AppData\Local\Temp\ipykernel_27128\1094039291.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[df.DiscoveryLink.isnull()]['DiscoveryLinks'] = df[df.DiscoveryLink.isnull()]['DoiPageLink'].apply(lambda doi_url: get_discovery_link_by_selenium(doi_url))


In [152]:
for i, row in df.iterrows():
    if row['DiscoveryLink'] is None:
        row['DiscoveryLink'] = get_discovery_link_by_selenium(row['DoiPageLink'])

[WDM] - ====== WebDriver manager ======


2023-02-23 22:40:37,239 INFO ====== WebDriver manager ======


[WDM] - Current google-chrome version is 110.0.5481


2023-02-23 22:40:37,242 INFO Current google-chrome version is 110.0.5481


[WDM] - Get LATEST chromedriver version for 110.0.5481 google-chrome


...

2023-02-23 22:48:44,638 INFO Current google-chrome version is 110.0.5481


[WDM] - Get LATEST chromedriver version for 110.0.5481 google-chrome


2023-02-23 22:48:44,639 INFO Get LATEST chromedriver version for 110.0.5481 google-chrome


[WDM] - Driver [C:\Users\ChrisLam\.wdm\drivers\chromedriver\win32\110.0.5481.77\chromedriver.exe] found in cache


2023-02-23 22:48:44,705 INFO Driver [C:\Users\ChrisLam\.wdm\drivers\chromedriver\win32\110.0.5481.77\chromedriver.exe] found in cache
selenium handling: https://www2.bgs.ac.uk/nationalgeosciencedatacentre/citedData/catalogue/e729d509-a616-4787-bb41-44c1169685b6.html
Message: no such element: Unable to locate element: {"method":"link text","selector":"Discovery Link"}
  (Session info: chrome=110.0.5481.104)
Stacktrace:
Backtrace:
	(No symbol) [0x00AC37D3]
	(No symbol) [0x00A58B81]
	(No symbol) [0x0095B36D]
	(No symbol) [0x0098D382]
	(No symbol) [0x0098D4BB]
	(No symbol) [0x009C3302]
	(No symbol) [0x009AB464]
	(No symbol) [0x009C1215]
	(No symbol) [0x009AB216]
	(No symbol) [0x00980D97]
	(No symbol) [0x0098253D]
	GetHandleVerifier [0x00D3ABF2+2510930]
	GetHandleVerifier [0x00D68EC1+2700065]
	GetHandleVerifier [0x00D6C86C+2714828]
	GetHandleVerifier [0x00B73480+645344]
	(No symbol) [0x00A60FD2]
	(No symbol) [0x00A66C68]
	(No symbol) [0x00A66D4B]
	(No symbol) [0x00A70D6B]
	BaseThreadInitThu